<a href="https://colab.research.google.com/github/RomainHerault/PRD/blob/master/Version_refactoris%C3%A9e.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
import torch
import numpy as np
import copy
import datetime
import os
import shutil
import cv2
import re
import torch.nn.functional as F
from fluideDataset import fluideDataset
from R2Plus1D import R2Plus1DClassifier

class Classifier:
  def __init__(self, number_classes):
    self.model = R2Plus1DClassifier(number_classes, (2, 2, 2, 2), pretrained=False).cuda()

  def load_model(self, path_model=None):
    if path_model is not None and os.path.exists(path_model):
      print('Pretrained Model existed')
      bestModelWts = torch.load(path_model)
      self.model.load_state_dict(bestModelWts)
    else:
      print("Le modèle n'est pas entraîné")
      bestModelWts = copy.deepcopy(self.model.state_dict()) # Copier les paramètres initiaux du modèle 

  def batch_predict(self, clip):
      outputs = self.model(clip.cuda().float())
      probs = F.softmax(outputs, dim=1)
      return probs.detach().cpu().numpy()

class Input_processing:
  def __init__(self, video_path):
    self.video_path = video_path

  def store_video_in_one_folder(self, number_images):
    numImages = 0
    path = self.video_path
    self.temp_video_folder = os.path.basename(path)+"_video"
    if not os.path.isdir(self.temp_video_folder):
      os.mkdir(self.temp_video_folder)
    sub_folders = [f.path for f in os.scandir(path) if f.is_dir()]
    sub_folders.sort(key=lambda f: int(re.sub('\D', '', f)))
    for filepath in (sub_folders):
      numFolder = int(os.path.basename(filepath))
      files = os.listdir(filepath)
      files.sort(key=lambda f: int(re.sub('\D', '', f)))
      for filename in files:
        base, extension = os.path.splitext(filename)
        num_file = numFolder * 30 + int(base)
        if(numImages<num_file):
          numImages = num_file
        new_name =  os.path.join(self.temp_video_folder,  str(num_file)+extension)
        shutil.copy(os.path.join(filepath,filename), new_name)
        if num_file + 1 >= number_images:
          return self.temp_video_folder
    return self.temp_video_folder

  def calcDoG(image):
    dogImg = np.empty((image.shape[0],image.shape[1],0))
    dog1 = -cv2.GaussianBlur(image,(0,0),scaleFactor[0])+image
    dog1[dog1<0]=0
    dog1 = np.expand_dims(dog1,axis = 2)
    dogImg = dogImg = np.concatenate((dogImg,dog1),axis = 2)
    for k in range(len(scaleFactor)-1):
      blur1 = cv2.GaussianBlur(image,(0,0),scaleFactor[k])
      blur2 = cv2.GaussianBlur(image,(0,0),scaleFactor[k+1])
      tempDoG = blur1 - blur2 
      tempDoG[tempDoG<0]=0
      tempDoG = np.expand_dims(tempDoG,axis = 2)
      dogImg = np.concatenate((dogImg,tempDoG),axis = 2)
    return dogImg

  def convert_folder_to_clip(videoFolder, seqLength):
    """
    Convert the png files to a clip
    videoFolder : the folder that contains all the png files
    seqLength : the number of images we want to take to create the clip 
    (the function takes the first n images)
    """
    filenames = []
    for filename in os.listdir(videoFolder):
      base, extension = os.path.splitext(filename)
      filenames.append(int(base))
    filenames.sort()
    clip = np.empty((0,96,256,5))
    for img_num in filenames :
      image = cv2.imread(videoFolder+"/"+str(img_num)+extension,0).astype(np.float64)/255.0
      image = cv2.resize(image, (256,96),interpolation=cv2.INTER_AREA)
      dogImg = Input_proccessing.calcDoG(image)
      dogImg = np.expand_dims(dogImg,axis = 0)
      clip = np.concatenate((clip,dogImg),axis = 0)
      if(img_num == seqLength):
        break
    clip = torch.from_numpy(np.array(clip.transpose([3,0,1,2])))
    print(clip.shape)
    return clip.unsqueeze(0)

  #print(batch_predict(convertFolderToClip(tempVideoFolder,15)))

  def convert_images_to_clip(images_list):
    """
    Convert the images files to a clip
    videoFolder : the folder that contains all the png files
    seqLength : the number of images we want to take to create the clip 
    (the function takes the first n images)
    """
    clip = np.empty((0,96,256,5))
    for image in images_list :
      image =  image.astype(np.float64)/255.0
      image = cv2.resize(image, (256,96),interpolation=cv2.INTER_AREA)
      dogImg = Input_processing.calcDoG(image)
      dogImg = np.expand_dims(dogImg,axis = 0)
      clip = np.concatenate((clip,dogImg),axis = 0)
    clip = torch.from_numpy(np.array(clip.transpose([3,0,1,2])))
    return clip.unsqueeze(0)

  def create_images(self):
    images_list = []#liste des segments de chaque image
    #Pour chaque image du dossier
    for f in os.listdir(self.temp_video_folder) :
      im = cv2.imread(self.temp_video_folder+"/"+f,0)
      images_list.append(im)
    return images_list

class Segmentation:
  installed = False
  def __init__(self):
    if not Segmentation.installed:
      !apt-get install build-essential cmake libboost-all-dev
      !git clone https://github.com/davidstutz/hierarchical-graph-based-video-segmentation.git
      !pwd
      %cd hierarchical-graph-based-video-segmentation
      !pwd
      !mkdir build
      %cd build
      !pwd
      !cmake ..
      !make
      %cd ..
      %cd ..
      Segmentation.installed = True

  def segment_images(self, video_folder):
    temp_flow_folder = os.path.basename(video_folder)+"_flow"
    if not os.path.isdir(temp_flow_folder):
      os.mkdir(temp_flow_folder)
      temp_out_folder = os.path.basename(video_folder)+"_out"
      if not os.path.isdir(temp_out_folder):
        os.mkdir(temp_out_folder)
      self.temp_vis_folder = os.path.basename(video_folder)+"_vis"
      if not os.path.isdir(self.temp_vis_folder) :
        os.mkdir(self.temp_vis_folder)
      num_images = os.listdir(video_folder)
      !./hierarchical-graph-based-video-segmentation/build/optical_flow_cli/optical_flow_cli --input-dir '$video_folder' --output-dir $temp_flow_folder
      !rm /content/$video_folder/0.png
      !./hierarchical-graph-based-video-segmentation/build/segment_cli/segment_cli '$video_folder' $temp_flow_folder --length $num_images --hierarchies 50 --vis-dir $self.temp_vis_folder --output-dir $temp_out_folder --input-gt ""
  
  def get_segments(self, segmentation_hierarchie):
    visualisation_folder = self.temp_vis_folder+'/'+str(segmentation_hierarchie)
    dico = dict()
    current_label = 0
    segments_list = []#liste des segments de chaque image
    #un segment = tableau de la taille de l'image contenant
    #le label du segmentu pour chaque pixel

    #Pour chaque image du dossier
    for f in os.listdir(visualisation_folder) :
      im = cv2.imread(visualisation_folder+"/"+f)
      #On créé un segment en labélisant les couleurs de l'image
      segment = np.zeros((im.shape[0], im.shape[1]))
      for x in range(im.shape[0]):
        for y in range(im.shape[1]):
          couleur = tuple(im[x,y])
          if (couleur not in dico):
            current_label += 1
            dico[couleur] = current_label
          segment[x,y] = dico[couleur]
      segments_list.append(segment)
    self.nb_segments = current_label
    return segments_list




In [0]:
!pip install lime

from lime import lime_image
from lime.wrappers.scikit_image import SegmentationAlgorithm
from skimage.segmentation import mark_boundaries
"""
Functions for explaining classifiers that use Video data.
"""
import copy
from functools import partial

import numpy as np
import sklearn
import sklearn.preprocessing
from sklearn.utils import check_random_state
from skimage.color import gray2rgb, rgb2gray
from progressbar import ProgressBar
from lime import lime_base
from google.colab.patches import cv2_imshow



class VideoExplanation(object):
    def __init__(self, images, segments):
        """Init function.

        Args:
            image: 3d numpy array
            segments: 2d numpy array, with the output from skimage.segmentation
        """
        self.images = images
        self.segments = segments
        self.intercept = {}
        self.local_exp = {}
        self.local_pred = None

    def get_images_and_masks(self, label, positive_only=True, negative_only=False, hide_rest=False,
                           num_features=5, min_weight=0.):
        """Init function.

        Args:
            label: label to explain
            positive_only: if True, only take superpixels that positively contribute to
                the prediction of the label.
            negative_only: if True, only take superpixels that negatively contribute to
                the prediction of the label. If false, and so is positive_only, then both
                negativey and positively contributions will be taken.
                Both can't be True at the same time
            hide_rest: if True, make the non-explanation part of the return
                image gray
            num_features: number of superpixels to include in explanation
            min_weight: minimum weight of the superpixels to include in explanation

        Returns:
            (image, mask), where image is a 3d numpy array and mask is a 2d
            numpy array that can be used with
            skimage.segmentation.mark_boundaries
        """
        if label not in self.local_exp:
            raise KeyError('Label not in explanation')
        if positive_only & negative_only:
            raise ValueError("Positive_only and negative_only cannot be true at the same time.")
        segments = self.segments
        images = self.images
        exp = self.local_exp[label]
        temps = []
        masks = []
        for i in range(len(images)):
          image = images[i]
          segment = segments[i]
          mask = np.zeros(segment.shape, dtype=int)
          if hide_rest:
              temp = np.zeros(image.shape)
          else:
              temp = image.copy()
          if positive_only:
              fs = [x[0] for x in exp
                    if x[1] > 0 and x[1] > min_weight][:num_features]
          if negative_only:
              fs = [x[0] for x in exp
                    if x[1] < 0 and abs(x[1]) > min_weight][:num_features]
          if positive_only or negative_only:
              for f in fs:
                  temp[segment == f] = image[segment == f].copy()
                  mask[segment == f] = 1
              temps.append(temp)
              masks.append(mask)
          else:
              for f, w in exp[:num_features]:
                  if np.abs(w) < min_weight:
                      continue
                  c = 0 if w < 0 else 1
                  mask[segment == f] = -1 if w < 0 else 1
                  temp[segment == f] = image[segment == f].copy()
                  temp[segment == f, c] = np.max(image)
              temps.append(temp)
              masks.append(mask)
        return temps, masks


class LimeVideoExplainer(object):
    """Explains predictions on Image (i.e. matrix) data.
    For numerical features, perturb them by sampling from a Normal(0,1) and
    doing the inverse operation of mean-centering and scaling, according to the
    means and stds in the training data. For categorical features, perturb by
    sampling according to the training distribution, and making a binary
    feature that is 1 when the value is the same as the instance being
    explained."""

    def __init__(self, kernel_width=.25, kernel=None, verbose=False,
                 feature_selection='auto', random_state=None):
        """Init function.
        
        Args:
            kernel_width: kernel width for the exponential kernel.
            If None, defaults to sqrt(number of columns) * 0.75.
            kernel: similarity kernel that takes euclidean distances and kernel
              width as input and outputs weights in (0,1). If None, defaults to
              an exponential kernel.
            verbose: if true, print local prediction values from linear model
            feature_selection: feature selection method. can be
              'forward_selection', 'lasso_path', 'none' or 'auto'.
              See function 'explain_instance_with_data' in lime_base.py for
              details on what each of the options does.
            random_state: an integer or numpy.RandomState that will be used to
              generate random numbers. If None, the random state will be
              initialized using the internal numpy seed.
        """
        kernel_width = float(kernel_width)
        
        if kernel is None:
            def kernel(d, kernel_width):
                return np.sqrt(np.exp(-(d ** 2) / kernel_width ** 2))
        
        kernel_fn = partial(kernel, kernel_width=kernel_width)
        
        self.random_state = check_random_state(random_state)
        self.feature_selection = feature_selection
        self.base = lime_base.LimeBase(kernel_fn, verbose, random_state=self.random_state)

    def explain_instance(self, images_list, classifier_fn, labels=(1,),
                        hide_color=None,
                        top_labels=5, num_features=100000, num_samples=1000,
                        batch_size=10,
                        segments=None,
                        distance_metric='cosine',
                        model_regressor=None,
                        random_seed=None):
        """Generates explanations for a prediction.
        
        First, we generate neighborhood data by randomly perturbing features
        from the instance (see __data_inverse). We then learn locally weighted
        linear models on this neighborhood data to explain each of the classes
        in an interpretable way (see lime_base.py).
        
        Args:
          image: 3 dimension RGB image. If this is only two dimensional,
              we will assume it's a grayscale image and call gray2rgb.
          classifier_fn: classifier prediction probability function, which
              takes a numpy array and outputs prediction probabilities.  For
              ScikitClassifiers , this is classifier.predict_proba.
          labels: iterable with labels to be explained.
          hide_color: TODO
          top_labels: if not None, ignore labels and produce explanations for
              the K labels with highest prediction probabilities, where K is
              this parameter.
          num_features: maximum number of features present in explanation
          num_samples: size of the neighborhood to learn the linear model
          batch_size: TODO
          distance_metric: the distance metric to use for weights.
          model_regressor: sklearn regressor to use in explanation. Defaults
          to Ridge regression in LimeBase. Must have model_regressor.coef_
          and 'sample_weight' as a parameter to model_regressor.fit()
          segmentation_fn: SegmentationAlgorithm, wrapped skimage
          segmentation function
          random_seed: integer used as random seed for the segmentation
              algorithm. If None, a random integer, between 0 and 1000,
              will be generated using the internal random number generator.
        
        Returns:
          An ImageExplanation object (see lime_image.py) with the corresponding
          explanations.
        """
        images = []
        for image in images_list:
            if len(image.shape) == 2:
                images.append(gray2rgb(image))
            else:
                images.append(image)
        
        images = np.array(images)
        
        if random_seed is None:
            random_seed = self.random_state.randint(0, high=1000)
        
        """
        if segmentation_fn is None:
            segmentation_fn = SegmentationAlgorithm('quickshift', kernel_size=4,
                                                  max_dist=200, ratio=0.2,
                                                  random_seed=random_seed)
        try:
            segments = segmentation_fn(image)
        except ValueError as e:
            raise e
        """
        fudged_image_list = []
        for image in images :
            fudged_image = image.copy()
            if hide_color is None:
                for x in np.unique(segments):#TODO
                    fudged_image[segments == x] = (
                        np.mean(image[segments == x][:, 0]),
                        np.mean(image[segments == x][:, 1]),
                        np.mean(image[segments == x][:, 2]))
            else:
                fudged_image[:] = hide_color
            fudged_image_list.append(fudged_image)
        
        top = labels
        data, labels = self.data_labels(images, fudged_image_list, segments,
                                      classifier_fn, num_samples,
                                      batch_size=batch_size)
        
        distances = sklearn.metrics.pairwise_distances(
            data,
            data[0].reshape(1, -1),
            metric=distance_metric
        ).ravel()
        
        
        ret_exp = VideoExplanation(images, segments)
        if top_labels:
            top = np.argsort(labels[0])[-top_labels:]
            ret_exp.top_labels = list(top)
            ret_exp.top_labels.reverse()
        for label in top:
            (ret_exp.intercept[label],
                ret_exp.local_exp[label],
                ret_exp.score, ret_exp.local_pred) = self.base.explain_instance_with_data(
                data, labels, distances, label, num_features,
                model_regressor=model_regressor,
                feature_selection=self.feature_selection)
        return ret_exp
        
        #return data, labels

    def data_labels(self,
                    images,
                    fudged_images,
                    segments,
                    classifier_fn,
                    num_samples,
                    batch_size=10):
        """Generates images and predictions in the neighborhood of this image.

        Args:
            image: 3d numpy array, the image
            fudged_image: 3d numpy array, image to replace original image when
                superpixel is turned off
            segments: segmentation of the image
            classifier_fn: function that takes a list of images and returns a
                matrix of prediction probabilities
            num_samples: size of the neighborhood to learn the linear model
            batch_size: classifier_fn will be called on batches of this size.

        Returns:
            A tuple (data, labels), where:
                data: dense num_samples * num_superpixels
                labels: prediction probabilities matrix
        """
        #n_features = np.unique(segments).shape[0]
        n_features = nb_segments

        data = self.random_state.randint(0, 2, num_samples * n_features)\
            .reshape((num_samples, n_features))
        labels = []
        data[0, :] = 1
        neighboorhood = []
        pbar = ProgressBar(num_samples)
        pbar.start()
        for row in data:
            temp_img_list = copy.deepcopy(images)
            zeros = np.where(row == 0)[0]
            temp_2d_img_list = []
            for i in range(len(images)):
                mask = np.zeros(segments[i].shape).astype(bool)
                for z in zeros:
                    mask[segments[i] == z] = True
                temp_img_list[i][mask] = fudged_images[i][mask]
                temp_2d_img_list.append(rgb2gray(temp_img_list[i]))
            neighboorhood.append(temp_2d_img_list)
            if len(neighboorhood) == batch_size:
                for neighboor in neighboorhood:
                    preds = classifier_fn(Input_processing.convertImagesToClip(neighboor))
                    labels.extend(preds)
                neighboorhood = []
            pbar.currval += 1
            pbar.update()
        pbar.finish()
        if len(neighboorhood) > 0:
            for neighboor in neighboorhood:
                preds = classifier_fn(Input_processing.convertImagesToClip(neighboor))
                labels.extend(preds)
        return data, np.array(labels)

#Execution

In [0]:
import matplotlib.pyplot as plt

classifier = Classifier(9)
model_path = "drive/My Drive/PRD/bestModelWts_LocMax060219.pt"
classifier.load_model(path_model = model_path)

video_path = 'drive/My Drive/PRD/automobile-engine-rotation-velocity-master/train/500_8_1_Gly'
input_processor = Input_processing(video_path=video_path)

video_folder = input_processor.store_video_in_one_folder(31)
print(video_folder)
images_list = input_processor.create_images()

segmenter = Segmentation()


In [0]:
segmenter.segment_images(video_folder)


In [0]:
"""
segments_list = segmenter.get_segments(segmentation_hierarchie = 35)

explainer = LimeVideoExplainer(verbose=False)
explanation = explainer.explain_instance(np.array(images_list), 
                                         batch_predict, # classification function
                                         batch_size = 10,
                                         top_labels=5, 
                                         hide_color=0,
                                         segments = segments_list,
                                         num_samples=10, # number of images that will be sent to classification function
                                         random_seed=1)

temps, masks = explanation.get_images_and_masks(explanation.top_labels[0], positive_only=False, num_features=1, hide_rest=False)
for temp, mask in zip(temps,masks):
  img_boundry1 = mark_boundaries(temp/255.0, mask)
  plt.figure()
  plt.imshow(img_boundry1)
  """